##**TASK-14**##

In [1]:
#Make required folders

import os

folders = ["raw", "processed", "output"]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created successfully")


Folders created successfully


In [4]:
#Load dataset
import pandas as pd

df = pd.read_csv("retail_sales_dataset.csv")
df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [5]:
import sqlite3

conn = sqlite3.connect("output/retail_etl.db")


In [6]:
# Check missing values
df.isnull().sum()



Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

In [7]:
# Remove duplicates
df_cleaned = df.drop_duplicates()


In [10]:
#Handle datatypes

df_cleaned.columns = (
    df_cleaned.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)


In [11]:
# Convert date column to datetime format
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"])


In [12]:
# Create a calculated total (validation column)
df_cleaned["calculated_total"] = (
    df_cleaned["quantity"] * df_cleaned["price_per_unit"]
)


In [13]:
# Flag high-value orders
df_cleaned["high_value_order"] = df_cleaned["total_amount"] > 1000


In [18]:
customers = df_cleaned[
    ["customer_id", "gender", "age"]
].drop_duplicates()



In [15]:
products = df_cleaned[
    ["product_category", "price_per_unit"]
].drop_duplicates()


In [16]:
orders = df_cleaned[
    [
        "transaction_id",
        "date",
        "customer_id",
        "product_category",
        "quantity",
        "total_amount",
        "high_value_order"
    ]
]


In [21]:
# Create SQLite database connection
conn = sqlite3.connect("output/retail_etl.db")

# Load tables into SQLite
customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)

# Close connection
conn.close()

print("Data loaded into SQLite database successfully")


Data loaded into SQLite database successfully


In [19]:
# Save processed datasets to output folder
customers.to_csv("output/customers.csv", index=False)
products.to_csv("output/products.csv", index=False)
orders.to_csv("output/orders.csv", index=False)

print("CSV files exported successfully")


CSV files exported successfully


In [20]:
# Validation: record counts
print("Total raw records:", len(df))
print("Total cleaned records:", len(df_cleaned))
print("Orders table records:", len(orders))
print("Customers table records:", len(customers))
print("Products table records:", len(products))


Total raw records: 1000
Total cleaned records: 1000
Orders table records: 1000
Customers table records: 1000
Products table records: 15


## INTERVIEW QUESTIONS

## 1. What is ETL and why is it needed?
    ETL stands for Extract, Transform, Load. It is used to collect data from raw sources, clean and standardize it, and load it into a structured format     such as a database or data warehouse. ETL is needed to ensure data quality, consistency, and reliability for analytics and decision-making.

## 2. ETL vs ELT?
    ETL: Data is transformed before loading into the target system. Common in traditional data warehouses.
    ELT: Data is loaded first and transformed inside the target system. Common in modern cloud platforms.
    ETL is preferred when data quality and structure must be enforced early in the pipeline.

## 3. How do you validate ETL output?
    ETL output is validated by:
     Comparing record counts before and after transformation
     Checking for null values and duplicates
     Verifying data types and derived columns
     Ensuring no unintended data loss during transformation

## 4. Why separate tables in an ETL pipeline?
    Separating tables (customers, products, orders) improves data normalization, reduces redundancy, and makes querying more efficient. It also aligns       with relational database design and supports scalable analytics.

## 5. What are common ETL errors?
    Common ETL errors include:
      Duplicate records
      Incorrect data types
      Missing or null values
      Schema mismatches